### Data Augmentation

We need to have our folder with the base training and another folder where we will store the transormed images

In [3]:
import os
import cv2 as cv
from tqdm import tqdm
from helpers import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

In [4]:
BASE_TRAINING = './base_training/'
BASE_TRAIN_IMAGES = BASE_TRAINING + 'images/'
BASE_TRAIN_GROUNDTRUTH = BASE_TRAINING + 'groundtruth/'

TRAINING = './training/'
TRAIN_IMAGES = TRAINING + 'images/'
TRAIN_GROUNDTRUTH = TRAINING + 'groundtruth/'

### Add 90, 180 and 270º rotations

In [6]:
%run helpers.py # To refresh the helpers file if there is an error

In [7]:
# Generate the rotated images
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_rotations = get_rotations_0_90_180_270(image)
    for i, rotated_image in enumerate(img_train_rotations[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_rotation_' + str((i+1)*90) + ".png"), rotated_image)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 12.94it/s]


In [8]:
# Generate the rotated groundtruths
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    img_train_rotations = get_rotations_0_90_180_270(image)
    for i, rotated_image in enumerate(img_train_rotations[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_rotation_' + str((i+1)*90) + ".png"), rotated_image)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.68it/s]


### Add randomly rotated images

In [9]:
%run helpers.py # To refresh the helpers file if there is an error

In [ ]:
# Pick how many rotations you want per image, 
# if you want more rotations delete all images and change the number here to avoid name conflicts
ROTATIONS_PER_IMAGE = 5
degrees = []
centers = []
for i in range(ROTATIONS_PER_IMAGE*len(os.listdir(BASE_TRAIN_IMAGES))):
    degrees.append(random.randint(0, 90)) # random rotation of degree between 0 and 90
    centers.append((random.randint(40, 60), random.randint(40, 60))) # random center between 40% and 60% of image width & length

In [ ]:
# Generate the rotated images
index = 0
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    for i in range(ROTATIONS_PER_IMAGE):
        random_rotation = get_rotation_deg_n(image, degrees[index], centers[index])
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_random_rotation_' + str(i) + ".png"), random_rotation)
        index += 1

In [ ]:
# Generate the rotated groundtruths
index = 0
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    for i in range(ROTATIONS_PER_IMAGE):
        random_rotation = get_rotation_deg_n(image, degrees[index], centers[index])
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_random_rotation_' + str(i) + ".png"), random_rotation)
        index += 1

### Add horizontally and vertically flipped images

In [ ]:
%run helpers.py # To refresh the helpers file if there is an error

In [ ]:
flip_type = ['x', 'y'] # Types of wanted flips

In [ ]:
# Generate the flipped images
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_flips = get_flipped_images(image)
    for i, flipped_image in enumerate(img_train_flips[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_flipped_' + flip_type[i] + ".png"), flipped_image)

In [ ]:
# Generate the flipped groundtruths
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    img_train_flips = get_flipped_images(image)
    for i, flipped_image in enumerate(img_train_flips[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_flipped_' + flip_type[i] + ".png"), flipped_image)

### Add gaussian noise

In [ ]:
%run helpers.py # To refresh the helpers file if there is an error

In [ ]:
# Variance of the gaussian noise
VARIANCE = 150

In [ ]:
# Generate the noisy images
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_noise = noisy('gauss', image, var=VARIANCE)
    cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_noise_' + 'gauss_var_' + str(VARIANCE) + ".png"), img_train_noise)

In [ ]:
# Copy the groundtruths as they do not change
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_noise_' + 'gauss_var_' + str(VARIANCE) + ".png"), image)

### Add salt and pepper noise

In [ ]:
%run helpers.py # To refresh the helpers file if there is an error

In [ ]:
# Ratio of pixels to be corrupted
CORRUPTION_RATIO = 0.02

In [ ]:
# Generate the noisy images
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_noise = noisy('s&p', image, var=VARIANCE)
    cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_noise_' + 's&p_corrupt_' + str(CORRUPTION_RATIO) + ".png"), img_train_noise)

In [ ]:
# Copy the groundtruths as they do not change
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_noise_' + 's&p_corrupt_' + str(CORRUPTION_RATIO) + ".png"), image)

### Add original images

In [ ]:
# Get the original images
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    cv.imwrite(os.path.join(TRAIN_IMAGES + img_name), image)

In [ ]:
# Get the original groundtruths
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH + img_name), image)

### Delete images

In [5]:
# Remove all images and groundtruths in the folders, do this whenever you want to try new augmentation sets
for filename in tqdm(os.listdir(TRAIN_IMAGES),total=len(os.listdir(TRAIN_IMAGES))):
    file_path = os.path.join(TRAIN_IMAGES, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
for filename in tqdm(os.listdir(TRAIN_GROUNDTRUTH),total=len(os.listdir(TRAIN_GROUNDTRUTH))):
    file_path = os.path.join(TRAIN_GROUNDTRUTH, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

100%|████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:00<00:00, 2155.99it/s]
